In [733]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import torch
from torch import nn

In [734]:
learning_rate = 0.1
batch_size = 2
epochs = 300

### read data

In [735]:
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')

In [736]:
np.unique(train['num_sold'])

array([  70,   73,   74, ..., 2807, 2851, 2884])

In [737]:
train.head()

,row_id,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
0,0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
1,1,0,0,0,520,2015,1,1,1,3,31,1,1,3,0
2,2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
3,3,0,1,1,572,2015,1,1,1,3,31,1,1,3,0
4,4,0,1,0,911,2015,1,1,1,3,31,1,1,3,0


### data preprocessing

In [738]:
train = train.set_index('row_id')
test = test.set_index('row_id')


In [739]:
# train = train[train['num_sold']<400]
# train = train[train['num_sold']>50]
# train = train[['country', 'store', 'product', 'month', 'year', 'num_sold']]

In [740]:
train

,country,store,product,num_sold,year,quarter,month,day,dayofweek,dayofmonth,dayofyear,weekofyear,weekday,is_weekend
row_id,,,,,,,,,,,,,,
0,0,0,1,329,2015,1,1,1,3,31,1,1,3,0
1,0,0,0,520,2015,1,1,1,3,31,1,1,3,0
2,0,0,2,146,2015,1,1,1,3,31,1,1,3,0
3,0,1,1,572,2015,1,1,1,3,31,1,1,3,0
4,0,1,0,911,2015,1,1,1,3,31,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,2,0,0,823,2018,4,12,31,0,31,365,1,0,0
26294,2,0,2,250,2018,4,12,31,0,31,365,1,0,0
26295,2,1,1,1004,2018,4,12,31,0,31,365,1,0,0


In [741]:
x_data = train.drop('num_sold', axis=1)
y_data = train.num_sold
std = StandardScaler()
x_data = std.fit_transform(x_data)
test = std.transform(test)

test = torch.tensor(test)
y_data = train[['num_sold']]
x_data = torch.tensor(x_data)
y_data = torch.tensor(y_data.values)

In [742]:
x_data

tensor([[-1.2247e+00, -1.0000e+00,  0.0000e+00,  ..., -1.7035e+00,
         -1.0264e-03, -6.3306e-01],
        [-1.2247e+00, -1.0000e+00, -1.2247e+00,  ..., -1.7035e+00,
         -1.0264e-03, -6.3306e-01],
        [-1.2247e+00, -1.0000e+00,  1.2247e+00,  ..., -1.7035e+00,
         -1.0264e-03, -6.3306e-01],
        ...,
        [ 1.2247e+00,  1.0000e+00,  0.0000e+00,  ..., -1.7035e+00,
         -1.5006e+00, -6.3306e-01],
        [ 1.2247e+00,  1.0000e+00, -1.2247e+00,  ..., -1.7035e+00,
         -1.5006e+00, -6.3306e-01],
        [ 1.2247e+00,  1.0000e+00,  1.2247e+00,  ..., -1.7035e+00,
         -1.5006e+00, -6.3306e-01]], dtype=torch.float64)

In [743]:
x_data = torch.tensor(x_data, dtype=torch.float32)
test = torch.tensor(test, dtype=torch.float32)

y_data = torch.tensor(y_data, dtype=torch.float32)

/tmp/ipykernel_4127245/1023721312.py:1: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/tmp/ipykernel_4127245/1023721312.py:2: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/tmp/ipykernel_4127245/1023721312.py:4: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



In [744]:
print(x_data[:10])

print(x_data.shape)

tensor([[-1.2247e+00, -1.0000e+00,  0.0000e+00, -1.3417e+00, -1.3504e+00,
         -1.6015e+00, -1.6738e+00, -1.0264e-03,  6.7963e-01, -1.7273e+00,
         -1.7035e+00, -1.0264e-03, -6.3306e-01],
        [-1.2247e+00, -1.0000e+00, -1.2247e+00, -1.3417e+00, -1.3504e+00,
         -1.6015e+00, -1.6738e+00, -1.0264e-03,  6.7963e-01, -1.7273e+00,
         -1.7035e+00, -1.0264e-03, -6.3306e-01],
        [-1.2247e+00, -1.0000e+00,  1.2247e+00, -1.3417e+00, -1.3504e+00,
         -1.6015e+00, -1.6738e+00, -1.0264e-03,  6.7963e-01, -1.7273e+00,
         -1.7035e+00, -1.0264e-03, -6.3306e-01],
        [-1.2247e+00,  1.0000e+00,  0.0000e+00, -1.3417e+00, -1.3504e+00,
         -1.6015e+00, -1.6738e+00, -1.0264e-03,  6.7963e-01, -1.7273e+00,
         -1.7035e+00, -1.0264e-03, -6.3306e-01],
        [-1.2247e+00,  1.0000e+00, -1.2247e+00, -1.3417e+00, -1.3504e+00,
         -1.6015e+00, -1.6738e+00, -1.0264e-03,  6.7963e-01, -1.7273e+00,
         -1.7035e+00, -1.0264e-03, -6.3306e-01],
        [-1.224

### model train

In [745]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data)

In [746]:
# np.unique(y_train.detach().numpy())

In [747]:
from torch.utils.data import TensorDataset

In [748]:
from torch.utils.data import DataLoader

In [749]:
dataset_train = TensorDataset(x_train, y_train)
loader_train = DataLoader(dataset_train, batch_size=256, shuffle=True)

dataset_test = TensorDataset(x_test, y_test)
loader_test = DataLoader(dataset_test, batch_size=256, shuffle=True)


In [750]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [751]:
import torch.nn.functional as F
n_feature = 13
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(n_feature, n_feature*2)
        self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        # self.bn1 = nn.BatchNorm1d(n_feature*4)
        # self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        # self.fc4 = nn.Linear(n_feature*8, n_feature*16)
        # # self.bn2 = nn.BatchNorm1d(n_feature*16)
        # self.fc5 = nn.Linear(n_feature*16, n_feature*32)
        # self.fc6 = nn.Linear(n_feature*32, n_feature*64)
        # self.fc7 = nn.Linear(n_feature*64, n_feature*128)
        # # self.bn3 = nn.BatchNorm1d(n_feature*128)
        # self.fc8 = nn.Linear(n_feature*128, n_feature*64)
        # self.fc9 = nn.Linear(n_feature*64, n_feature*32)
        # self.fc10 = nn.Linear(n_feature*32, n_feature*16)
        # # self.bn4 = nn.BatchNorm1d(n_feature*16)
        # self.fc11 = nn.Linear(n_feature*16, n_feature*8)
        # self.fc12 = nn.Linear(n_feature*8, n_feature*4)
        self.bn5 = nn.BatchNorm1d(n_feature*4)
        self.fc13 = nn.Linear(n_feature*4, n_feature*2)
        self.fc14 = nn.Linear(n_feature*2, n_feature)
        self.fc15 = nn.Linear(n_feature, 1)

        # self.fc1 = nn.Linear(n_feature, n_feature*2)
        # self.bn1 = nn.BatchNorm1d(n_feature*2)
        # self.fc2 = nn.Linear(n_feature*2, n_feature*4)
        # self.bn2 = nn.BatchNorm1d(n_feature*4)
        # self.fc3 = nn.Linear(n_feature*4, n_feature*8)
        # self.bn3 = nn.BatchNorm1d(n_feature*8)
        # self.fc4 = nn.Linear(n_feature*8, n_feature*4)
        # self.fc5 = nn.Linear(n_feature*4, n_feature*2)
        # self.bn5 = nn.BatchNorm1d(n_feature*2)
        # self.fc6 = nn.Linear(n_feature*2, n_feature)
        # self.fc7 = nn.Linear(n_feature,  1)


    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(self.fc2(x))
        # x = self.bn1(x)

        # x = F.relu(self.fc3(x))
        # x = F.relu(self.fc4(x))
        # # x = self.bn2(x)
        # x = F.relu(self.fc5(x)) 
        # x = F.relu(self.fc6(x) )
        # x = F.relu(self.fc7(x) )
        # # x = self.bn3(x) 
        # x = F.relu(self.fc8(x) )
        # x = F.relu(self.fc9(x) )
        # x = F.relu(self.fc10(x) )
        # # x = self.bn4(x)
        # x = F.relu(self.fc11(x) )
        # x = F.relu(self.fc12(x) )
        x = self.bn5(x) 
        x = F.relu(self.fc13(x) )
        x = F.relu(self.fc14(x))
        x = self.fc15(x) 
        
        # x = self.fc1(x)
        # x = F.relu(self.bn1(x))
        # x = F.relu(self.fc2(x))
        # x= F.relu(self.bn2(x) )
        # x= F.relu(self.fc3(x) )
        # x= F.relu(self.bn3(x) )
        # x= F.relu(self.fc4(x) )
        # x = F.relu(self.fc5(x))
        # x = F.relu(self.bn5(x))
        # x = F.relu(self.fc6(x))
        # x = self.fc7(x)
        # x = self.fc8(x)


        
        return x

In [752]:
model = NeuralNetwork().to(device)
# model.apply(init_weights)
print(model)

NeuralNetwork(
  (fc1): Linear(in_features=13, out_features=26, bias=True)
  (fc2): Linear(in_features=26, out_features=52, bias=True)
  (bn5): BatchNorm1d(52, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc13): Linear(in_features=52, out_features=26, bias=True)
  (fc14): Linear(in_features=26, out_features=13, bias=True)
  (fc15): Linear(in_features=13, out_features=1, bias=True)
)


In [753]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        
        X = X.to(device)
        y = y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        # scheduler.step()
        optimizer.step()

    loss = loss.item()
    loss = loss ** 0.5
    print(f"loss: {loss:>7f}")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    test_loss = test_loss ** 0.5
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

In [754]:
from math import exp
import math

In [755]:
loss_fn = nn.MSELoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.5)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# step_size = 4*len(dataset_train)
# clr = cyclical_lr(step_size)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.985)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(loader_train, model, loss_fn, optimizer)
    test_loop(loader_test, model, loss_fn)
    scheduler.step()
    print(optimizer.param_groups[0]["lr"])
print("Done!")

Epoch 1
-------------------------------
loss: 83.972867
Test Error: 
 Avg loss: 133.966860 

0.0985
Epoch 2
-------------------------------
loss: 92.155529
Test Error: 
 Avg loss: 123.886824 

0.0970225
Epoch 3
-------------------------------
loss: 94.228301
Test Error: 
 Avg loss: 167.659786 

0.0955671625
Epoch 4
-------------------------------
loss: 71.560204
Test Error: 
 Avg loss: 123.474893 

0.0941336550625
Epoch 5
-------------------------------
loss: 40.110247
Test Error: 
 Avg loss: 90.531049 

0.0927216502365625
Epoch 6
-------------------------------
loss: 86.379873
Test Error: 
 Avg loss: 76.207416 

0.09133082548301405
Epoch 7
-------------------------------
loss: 176.434389
Test Error: 
 Avg loss: 71.310193 

0.08996086310076884
Epoch 8
-------------------------------
loss: 123.736075
Test Error: 
 Avg loss: 103.277341 

0.08861145015425731
Epoch 9
-------------------------------
loss: 128.552974
Test Error: 
 Avg loss: 82.783543 

0.08728227840194346
Epoch 10
----------

In [756]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)


In [757]:
x_train = x_train.to(device)
x_test = x_test.to(device)

train_pred = model(x_train)
test_pred = model(x_test)

train_score = SMAPE(y_train.cpu().detach().numpy(), train_pred.cpu().detach().numpy())
test_score = SMAPE(y_test.cpu().detach().numpy(), test_pred.cpu().detach().numpy())

print('train_score ', train_score, 'test_score', test_score)

train_score  5.4865584 test_score 5.802652


In [758]:
test = test.to(device)

In [759]:
test_pred = model(test)

In [760]:
train_pred.cpu()

tensor([[168.1647],
        [124.5457],
        [221.0626],
        ...,
        [279.1246],
        [505.3298],
        [198.0134]], grad_fn=<AddmmBackward0>)

In [761]:
# unique, counts = np.unique(y_train.cpu().detach().numpy(), return_counts=True)
# unique_p, counts_p = np.unique(np.round(train_pred.cpu().detach().numpy()), return_counts=True)
# d_p = pd.DataFrame()
# d_p.index = unique_p
# d_p['count_pred'] = counts_p
# d = pd.DataFrame()
# d.index = unique
# d['count_true'] = counts
# d1 = d.join(d_p)
# d1 = d1.fillna(0)


In [762]:
unique, counts = np.unique(y_train.cpu().detach().numpy(), return_counts=True)
unique_p, counts_p = np.unique(np.round(train_pred.cpu().detach().numpy()), return_counts=True)
d_p = pd.DataFrame(index = unique_p,data = counts_p,columns=['count_pred'])
d = pd.DataFrame(index= unique,data=  counts,columns=['count_true'])
d1 = d.join(d_p)
d1 = d1.fillna(0)

In [763]:
import plotly.express as px

fig = px.line(d1, x= d.index, y = ['count_true', 'count_pred'])
fig.show()

In [764]:
submission = pd.read_csv('sample_submission.csv')
submission.num_sold = test_pred.cpu().detach().numpy().reshape(-1)
submission.to_csv('submission.csv', index=False)